In [2]:
using Luxor
using Colors
using Plots
using IterTools
using DataFrames
using OpenStreetMapX
using LightOSM
using KernelDensity
using Parsers
using Downloads
using OSMToolset
include("kernel_density.jl")
include("distance.jl")
include("prepare_data.jl")
include("analyse.jl")
include("plots.jl")

plot_attractiveness_of_sectors_prcnt (generic function with 1 method)

In [4]:
outfiles = tile_osm_file("Kielce.osm",nrow=2,ncol=2,out_dir="./tiles")

2×2 Matrix{String}:
 "50.7932894_50.854186_20.5105231_20.6155755.osm"  …  "50.7932894_50.854186_20.6155755_20.7206279.osm"
 "50.854186_50.9150826_20.5105231_20.6155755.osm"     "50.854186_50.9150826_20.6155755_20.7206279.osm"

In [14]:
function tile_road_length(city_parse_data,city_centre)
    road_types = ["motorway", "trunk", "primary", "secondary", 
                "tertiary", "residential", "service", "living_street", 
                "motorway_link", "trunk_link", "primary_link", "secondary_link", 
                "tertiary_link"]
    
    total_length = 0.0
    
    for way in city_parse_data.ways
        if haskey(way.tags, "highway") && (way.tags["highway"] in road_types)
            for i in 1:(length(way.nodes) - 1)
                node1 = ENU(city_parse_data.nodes[way.nodes[i]],city_centre)
                node2 = ENU(city_parse_data.nodes[way.nodes[i + 1]],city_centre)
                total_length += OpenStreetMapX.distance(node1,node2)
            end
        end
    end
    
    return total_length
end

tile_road_length (generic function with 2 methods)

In [7]:
dir = "tiles/"
elem = readdir(dir)[1]
file = joinpath(dir,elem)
city_parse = OpenStreetMapX.parseOSM(file)

OpenStreetMapX.OSMData(Dict{Int64, LLA}(417920770 => LLA(50.850966, 20.5476277, 0.0), 3590772795 => LLA(50.826289, 20.5240614, 0.0), 3590629085 => LLA(50.8244846, 20.5277449, 0.0), 2317905797 => LLA(50.8327167, 20.557977, 0.0), 9730625255 => LLA(50.8249517, 20.5661247, 0.0), 10236048654 => LLA(50.853586, 20.5139479, 0.0), 4205780925 => LLA(50.7946632, 20.5859509, 0.0), 9026126158 => LLA(50.8112352, 20.5963557, 0.0), 9730625167 => LLA(50.8162359, 20.5671308, 0.0), 11413702094 => LLA(50.8409973, 20.5349883, 0.0)…), OpenStreetMapX.Way[OpenStreetMapX.Way(35710973, [417877646, 8972017067, 2582317378, 3417499545, 3417499505, 3417499548, 3417499519, 8972017061, 8972017029, 8972017025, 8972017055, 8972017018, 4458162659, 417878428], Dict("name" => "Składowa", "source" => "Bing", "highway" => "residential")), OpenStreetMapX.Way(35711095, [417920402, 9386312514, 9386312365, 9386312368, 417920159], Dict("source" => "Bing", "highway" => "service")), OpenStreetMapX.Way(35711169, [10876012993, 10876

In [15]:
city_map = create_map("Kielce.osm")

┌ Info: Saved map data to cache Kielce.osm.cache
└ @ OpenStreetMapX /home/adamkas/.julia/packages/OpenStreetMapX/gCd33/src/parseMap.jl:110


MapData(OpenStreetMapX.Bounds{LLA}(50.7932894, 50.9150826, 20.5105231, 20.7206279), Dict{Int64, ENU}(10604990699 => ENU(-2588.701687179754, 6230.1428589448615, -3.5690860663858075), 417920770 => ENU(-4785.040710211535, -356.00989568734803, -1.801255370984677), 9375611775 => ENU(2406.473224601477, 1981.8307426852828, -0.7611705030562916), 9730625255 => ENU(-3484.376355259608, -3251.0093605463544, -1.778932156395058), 10544419049 => ENU(1775.4087909655814, 4674.877182856469, -1.9609709910519086), 2645603661 => ENU(529.48404427335, 4758.4917401685, -1.7981773198921474), 9063420015 => ENU(-5154.887418010016, 3078.455775296769, -2.8223334417796195), 11099755538 => ENU(198.675805103436, 4421.715058338904, -1.5368062112002008), 9499568987 => ENU(-6388.452595859622, -465.5447394046695, -3.2099479695323225), 6981579993 => ENU(-6627.1424150795365, -3134.1867728727584, -4.206579059730302)…), OpenStreetMapX.Way[OpenStreetMapX.Way(32808666, [1271853547, 2342136164, 1271853589, 2342136121, 127185366

In [17]:
city_centre = OpenStreetMapX.center(city_map.bounds)

LLA(50.854186, 20.6155755, 0.0)

In [18]:
tile_road_length(city_parse,city_centre)

214313.67156523792

In [14]:
mapp = create_map(file)

┌ Info: Read map data from cache tiles/50.7932894_50.854186_20.5105231_20.6155755.osm.cache
└ @ OpenStreetMapX /home/adamkas/.julia/packages/OpenStreetMapX/gCd33/src/parseMap.jl:97


MapData(OpenStreetMapX.Bounds{LLA}(50.7932894, 50.854186, 20.5105231, 20.6155755), Dict{Int64, ENU}(10834345591 => ENU(-2033.8125431560932, 3042.585304098181, -1.0498032071961916), 417920770 => ENU(-1086.0248635356438, 3029.1297948472475, -0.812058389281674), 9730625255 => ENU(216.6972499577557, 135.0558306722452, -0.005104582059360041), 11347204516 => ENU(-2007.1053950935855, 420.0471871301951, -0.3290081297299423), 2032319556 => ENU(-1990.4794721472954, 1336.0103632884118, -0.4499862381087496), 602886818 => ENU(-3570.8304502907213, 735.2853487317834, -1.0399713394211858), 3710912037 => ENU(2954.710539712472, 1997.7764913429774, -0.996098918114285), 5948028161 => ENU(-196.45085628957713, 3195.7214672375067, -0.8041519284067817), 4458103069 => ENU(-1944.247579697284, 842.0090571084925, -0.35135182855890434), 9730625441 => ENU(-380.04915380817795, -555.2314381684183, -0.035483348487616695)…), OpenStreetMapX.Way[OpenStreetMapX.Way(35710973, [417877646, 8972017067, 2582317378, 3417499545,

In [15]:
centre = OpenStreetMapX.center(mapp.bounds)

LLA(50.823737699999995, 20.5630493, 0.0)

In [16]:
llas = extract_points_LLA(file,centre)

Row,adminname,wayid,nodes,x,y
,String,Int64,Int64,Float64,Float64
1,Kielce,419975553,1327578229,20.6148,50.7969
2,Kielce,419975553,1327578311,20.6166,50.8009
3,Kielce,419975553,4199593809,20.6155,50.801
4,Kielce,419975553,4199645515,20.6155,50.801
5,Kielce,419975553,2291043305,20.6155,50.801
6,Kielce,419975553,1327578748,20.615,50.8055
7,Kielce,419975553,1327578651,20.6027,50.8102
8,Kielce,419975553,4199645626,20.6026,50.8102
9,Kielce,419975553,3590682494,20.6012,50.8095


In [17]:
llas

Row,adminname,wayid,nodes,x,y
,String,Int64,Int64,Float64,Float64
1,Kielce,419975553,1327578229,20.6148,50.7969
2,Kielce,419975553,1327578311,20.6166,50.8009
3,Kielce,419975553,4199593809,20.6155,50.801
4,Kielce,419975553,4199645515,20.6155,50.801
5,Kielce,419975553,2291043305,20.6155,50.801
6,Kielce,419975553,1327578748,20.615,50.8055
7,Kielce,419975553,1327578651,20.6027,50.8102
8,Kielce,419975553,4199645626,20.6026,50.8102
9,Kielce,419975553,3590682494,20.6012,50.8095


In [ ]:
admin_city_centre = get_city_centre(boundaries_file)

In [13]:
city.ways

14534-element Vector{OpenStreetMapX.Way}:
 OpenStreetMapX.Way(35710973, [417877646, 8972017067, 2582317378, 3417499545, 3417499505, 3417499548, 3417499519, 8972017061, 8972017029, 8972017025, 8972017055, 8972017018, 4458162659, 417878428], Dict("name" => "Składowa", "source" => "Bing", "highway" => "residential"))
 OpenStreetMapX.Way(35711095, [417920402, 9386312514, 9386312365, 9386312368, 417920159], Dict("source" => "Bing", "highway" => "service"))
 OpenStreetMapX.Way(35711169, [10876012993, 10876012991, 5651792186, 9243558533, 9243558532, 9243558531, 9243558530, 9243558529, 9243558528, 5651792187, 9243558527], Dict("highway" => "path"))
 OpenStreetMapX.Way(35711174, [2887890345, 417920737, 417923240, 417920738], Dict("name" => "Sławkowska", "highway" => "residential", "surface" => "asphalt"))
 OpenStreetMapX.Way(35711299, [9169495482, 10067047072, 4439601867, 10067047070, 9169495495, 10067047068, 4439601866, 10067047067, 9169495494, 9141536985, 10068587453, 3710912010, 417842091], 